**Modelos y Simulación - Primer cuatrimestre de 2024 - U.N.C**
**Alumna: Julieta Paola Storino**

# IMPORTANCE SAMPLING
## Introducción

El presente informe tiene como objetivo presentar un método para mejorar la eficiencia en la simulación de eventos raros, el método de Importance Sampling, que logra optimizar la convergencia de los estimadores de Monte Carlo. Para ello, se abordarán dos problemas de probabilidad utilizando tanto el método de Monte Carlo tradicional como el método de Importance Sampling, y se compararán los resultados obtenidos con ambos enfoques.

El primer problema consiste en aproximar la probabilidad $P(X>3)$, donde $X$ es una variable aleatoria con distribución normal estándar. Posteriormente, se resolverá el problema de calcular la probabilidad de que una línea de soporte técnico reciba al menos 9 llamadas en un periodo de 10 minutos.

## Algoritmo y descripción de las Variables:
### Problema 1

Sea $X$ una variable aleatoria con distribución normal y $h(t)=I_{(3,\infty)}(t)$ la función característica del intervalo $(3,\infty)$. Entonces, si desarrollamos la probabilidad de interés mediante Monte Carlo, debemos primero considerar una secuencia $X_i\sim N(0,1)$ de variables aleatorias independientes, luego $h(X_1), h(X_2), \ldots$ son variables aleatorias independientes e idénticamente distribuidas, con media igual a $E[g(X)]$, por la Ley Fuerte de los Grandes Números, se tiene que:

$$E[h(X)]\simeq \lim_{n\rightarrow\infty}\frac{1}{n}\sum_{i=1}^{n}h(X_i). \tag{1}$$

De esta manera, para calcular el valor de la probabilidad de interés, se propone generar una realización de $n$ variables aleatorias $X$, $x_1,...,x_n$, con distribución $N(0,1)$, para luego calcular la media de la función indicadora de que $x_i$ pertenece al intervalo $(3,\infty)$. Así, el algoritmo para resolver el problema con el método de Monte Carlos es el siguiente:

```python

def h(x): # Función indicadora del intervalo (3,inf)
    if x > 3: return 1
    else:     return 0

def monteCarlo(n):
    estimacion = 0
    for _ in range(n):
        Generar X
        estimacion += h(X)
    return estimacion/n

```

Por otro lado, para resolver el problema con el método de Importance Sampling, se utiliza una distribución de importancia $g_Y(x)$, que representa a la función de densidad de una variable aleatoria $Y\sim F$. Para ello, podemos reescribir (1) como:

$$
\begin{align}
E[h(X)] &= \int^{\infty}_{-\infty} h(t)f_X(t)dt = \int^{\infty}_{-\infty} h(t)f_X(t)\Big(\frac{g_Y(t)}{g_Y(t)}\Big)dt \tag{2}\\
&= \int^{\infty}_{-\infty} \frac{h(t)f_X(t)}{g_Y(t)}g_Y(t)dt \tag{3}\\
&= E\Big[\frac{h(Y)f_X(Y)}{g_Y(Y)}\Big] \tag{4},
\end{align}
$$

y siguiendo el mismo razonamiento que en (1), se tiene que:

$$E\Big[\frac{h(Y)f_X(Y)}{g_Y(Y)}\Big]\simeq \lim_{n\rightarrow\infty}\frac{1}{n}\sum_{i=1}^{n}\frac{h(Y_i)f_X(Y_i)}{g_Y(Y_i)}. \tag{5}$$

De esta manera, para calcular el valor de la probabilidad de interés, se realizan simulaciones con $10000, 20000, ..., 500000$ puntos de muestreo, utilizando tres distribuciones de importancia: 

$$g_Y(x) = N(4,1), \quad g_Y(x) = \mathcal{E}(1), \quad g_Y(x) = \text{Gamma}(9,1/2),$$

para poder comparar los resultados obtenidos y la eficiencia en la convergencia de cada una de ellas, con respecto al método de Monte Carlo tradicional. Así, suponiendo que sabemos generar y evaluar respecto a las tres distribuciones, el algoritmo para resolver el problema con el método de Importance Sampling es el siguiente:

```python

def f(x): # Función de densidad de la distribución normal estándar
    return (Y ** 8 * exp(-(Y * 2))) / (0.5 ** 9 * factorial(8))

def ImportanceSampling(n):
    estimacion = 0
    for _ in range(n):
        generar Y
        estimacion += h(Y) * f(Y) / g(Y)
    return estimacion / n

```

### Problema 2

Para dar solución al problema, primero hay que desarrollarlo. Sea $S_i$ la variable aleatoria que representa el *tiempo de espera hasta recibir la $i$-ésima llamada*, tal que $S_i \sim \text{Gamma}(i,1/2)$, podemos representar a la probabilidad de que tengan que esperar al menos $10$ minutos para recibir $9$ llamadas como $P(S_9\ge 10)$, con $S_9\sim \text{Gamma}(9,1/2)$. Entonces, la probabilidad de interés es:

$$
\begin{align}
P(S_9\ge 10) &= \int_{10}^{\infty}\frac{x^8e^{-(x * 2)}}{0.5^9\times\Gamma(9)}dx \tag{6} \\
&= \int_{-\infty}^{\infty}I_{[10,\infty)}(x)\frac{x^8e^{-(x * 2)}}{0.5^9\times8!}dx \tag{7} \\
&= E[I_{[10,\infty)}(X)] \tag{8} \\
&= E[h(X)]. \tag{9}
\end{align}
$$

Donde $h(x) = I_{[10,\infty)}(x)$ es la función indicadora de que $x$ pertenece al intervalo $[10,\infty)$. Luego, sean $X_i\sim\text{Gamma}(9,1/2)$, con $i=1,2,...,n$, variables aleatorias independientes e idénticamente distribuidas, para un $n$ suficientemente grande, y para cualquier realización de las variables aleatorias $X_i$, $ X_1 = x_1, X_2 = x_2, ..., X_n = x_n$, si desarrollando con Monte Carlos, se tiene que:

$$ P(S_9\ge 10) = \frac1n\sum_{i=1}^nh(x_i).$$

De esta manera, para calcular el valor de la probabilidad de interés, se propone generar una realización de $n$ variables aleatorias, con distribución $\text{Gamma}(9,1/2)$, para luego calcular la media de la función indicadora de que $x_i$ pertenece al intervalo $[10,\infty)$. Así, el algoritmo para resolver el problema con el método de Monte Carlos es el siguiente:

```python

def h(x): # Función indicadora del intervalo [10,inf)
    if x >= 10: return 1
    else:       return 0

def monteCarlo(n):
    estimacion = 0
    for _ in range(n):
        Generar X
        estimacion += h(X)
    return estimacion/n

```

Por otro lado, para resolver el problema con el método de Importance Sampling, se propone utilizar una distribución de importancia $g_Y(x)$, que representa a la función de densidad de una variable aleatoria $Y\sim N(11,1)$. Luego, sean $Y_i\sim N(11,1)$, con $i=1,2,...,n$, variables aleatorias independientes e idénticamente distribuidas, para un $n$ suficientemente grande, y para cualquier realización de las variables aleatorias $Y_i$, $ Y_1 = y_1, Y_2 = y_2, ..., Y_n = y_n$, si desarrollando con Importance Sampling, se tiene que la probabilidad de interés se puede reescribir, a partir de (4) como:

$$
\begin{align}
E[h(X)] &= \int_{-\infty}^{\infty}h(x)\frac{f(x)}{g_Y(x)}g_Y(x)dx \tag{5} \\
&= \int_{-\infty}^{\infty}h(x)\frac{f(x)}{g_Y(x)}g_Y(x)dx \tag{6} \\
&= E\left[\frac{h(Y)f(Y)}{g_Y(Y)}\right], \tag{7} \\
\end{align}
$$

De esta manera, para calcular el valor de la probabilidad de interés, se propone generar una realización de $n$ variables aleatorias, con distribución $N(11,1)$, para luego calcular la media de la función $h(Y)f(Y)/g_Y(Y)$. Así, el algoritmo para resolver el problema con el método de Importance Sampling es el siguiente:

```python

def f(x): # Función de densidad de la distribución Gamma(9,1/2)
    return (x ** 8 * exp(-2 * x)) / (0.5 ** 9 * factorial(8))


def g(x): # Función de densidad de la distribución N(11,1)
    return exp(- ((x - 11) ** 2) / 2) / sqrt(2 * pi)

def ImportanceSampling(n):
    estimacion = 0
    for _ in range(n):
        Generar Y
        estimacion += h(Y) * f(Y) / g(Y)
    return estimacion / n

```

## Resultados Obtenidos
### Problema 1

<div style="text-align: center;">
    <img src="https://i.ibb.co/qrL3Tw0/Primer-Ejercicio-MC.png" alt="Gráfico de la aproximación a una probabilidad normal por Monte Carlo">
    <p><em>Gráfico de la aproximación a una probabilidad normal por Monte Carlo</em></p>
</div>

<div style="text-align: center;">
    <img src="https://i.ibb.co/fYZLwZ9/Grafico-MCvs-IS.png" alt="Gráfico de la aproximación a una probabilidad normal por Importance Sampling">
    <p><em>Gráfico de la aproximación a una probabilidad normal por Importance Sampling</em></p>
</div>

### Problema 2

<div style="text-align: center;">
    <img src="https://i.ibb.co/NpTwPFN/Primer-Ejercio-IS.png" alt="Gráfico de P(S9 ≥ 10)">
    <p><em>Gráfico de P(S9 ≥ 10)</em></p>
</div>

## Conclusiones

Los resultados obtenidos demuestran que el método de Importance Sampling ofrece una mejora significativa en la eficiencia y precisión de las estimaciones en comparación con el método de Monte Carlo tradicional. En el primer problema, utilizando distribuciones de importancia como la normal $N(4,1)$, la exponencial $\mathcal{E}(1)$ y la Gamma $\text{Gamma}(9,1/2)$, se observó una convergencia más rápida y una menor varianza en las estimaciones, en contraste con las simulaciones de Monte Carlo. Similarmente, en el segundo problema, emplear una distribución de importancia normal $N(11,1)$ permitió obtener estimaciones más precisas de la probabilidad deseada, comparado con el método de Monte Carlo.

Lo que también se puede observar es que la elección de la distribución de importancia es crucial para el éxito del método de Importance Sampling ya que, en el primer problema, la distribución normal $N(4,1)$ resultó ser notablemente más eficaz que la exponencial $\mathcal{E}(1)$ y la Gamma $\text{Gamma}(9,1/2)$. Por otro lado, en el segundo problema, la distribución de importancia seleccionada también tuvo un impacto significativo en la eficiencia y precisión de las estimaciones. Por lo tanto, es fundamental elegir una distribución de importancia que se ajuste adecuadamente a la función objetivo para maximizar la eficacia del método.

En conclusión, el método de Importance Sampling se presenta como una herramienta valiosa y eficiente para la simulación de eventos raros, optimizando la convergencia de los estimadores y reduciendo la varianza en las estimaciones. La implementación de este método en problemas prácticos puede resultar en simulaciones más rápidas y precisas, siendo especialmente útil en aplicaciones donde la ocurrencia de eventos raros es crítica. 